In [9]:
import requests
from bs4 import BeautifulSoup

# 웹사이트 URL 설정
url = "https://www.ytn.co.kr/_ln/0106_202410290940108043"

# 요청 보내기
response = requests.get(url)

# 요청이 성공했는지 확인
if response.status_code == 200:
    # HTML 내용 파싱
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # 게시글 제목 가져오기 (예: 'h2' 태그)
    title = soup.find('h2')  # 제목이 포함된 태그를 수정하세요
    title_text = title.text.strip() if title else "제목을 찾을 수 없습니다."

    # 게시글 본문 가져오기 (예: 'div' 태그와 특정 클래스)
    content = soup.find('div', class_='article')  # 본문이 포함된 태그와 클래스를 수정하세요
    content_text = content.text.strip() if content else "본문을 찾을 수 없습니다."
    
    # 간단한 요약 생성 (여기서는 첫 100자 사용)
    summary = content_text[:100] + "..." if content_text else ""

    # 출력
    print(f"제목: {title_text}")
    print("본문:")
    print(content_text)
    print("\n요약:")
    print(summary)

else:
    print("웹사이트를 불러오는 데 실패했습니다.")


제목: 많이 본 뉴스
본문:
본문을 찾을 수 없습니다.

요약:
본문을 찾을 수 없습니다....


In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader  # 웹에서 문서를 로드하는 모듈
from langchain_text_splitters import RecursiveCharacterTextSplitter  # 텍스트를 재귀적으로 분할하는 모듈
from langchain_community.vectorstores import FAISS  # FAISS 벡터 저장소 모듈
from langchain_openai import OpenAIEmbeddings, ChatOpenAI  # OpenAI 임베딩 및 채팅 LLM 모듈
from langchain_core.runnables import RunnablePassthrough  # 파이프라인에서 입력을 그대로 전달하는 모듈
from langchain_core.output_parsers import StrOutputParser  # 문자열 출력을 처리하는 파서 모듈
from langchain_core.prompts import PromptTemplate  # 프롬프트 템플릿을 생성하는 모듈
from dotenv import load_dotenv  # 환경 변수를 로드하는 모듈

load_dotenv()

# 웹 페이지 로딩
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/003/0012872842?sid=104",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body","media_end_head go_trans","mdedia_end_summary"]}
        )
    )
)

# 문서 로드
docs = loader.load()
print("로딩된 문서 수:", len(docs))

# 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
splits = text_splitter.split_documents(docs)
print("생성된 스플릿 수:", len(splits))

# print(docs)

# 임베딩 생성 및 벡터 저장소 생성
if splits:
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(documents=splits, embedding=embeddings)
else:
    raise ValueError("문서에서 생성된 스플릿이 없습니다.")

retriver = vectorstore.as_retriever()

# 질문-답변용 프롬프트 설정
prompt = PromptTemplate.from_template(
"""
당신은 질문-답변을 수행하는 AI 어시스턴트이다.
주어진 문맥에 검색된 다음문맥(context)를 사용해 질문에 답하세요.
만약 주어진 문맥에서 답을 찾을 수 없을 경우, 모른다고 이야기하세요.
한글로 답변해주세요.

#Question
{question}

#Context:
{context}

#Answer:"""
)

# LLM 설정
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 질문-답변 체인 구성
news_chain = (
    {"context": retriver, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 질문 실행
try:
    answer = news_chain.invoke("일론 머스크")
    print(answer)
except Exception as e:
    print("호출 중 오류 발생:", str(e))


로딩된 문서 수: 1
생성된 스플릿 수: 21
머스크는 세계에서 인구 붕괴가 가장 큰 위협 사항이라고 언급하면서 "한국 봐라"라고 구체적인 사례로 든 것으로 보입니다. 그는 특히 한국의 출산율이 0.72명까지 감소하면서 역대 최저치를 기록한 점과 한국 인구가 현재의 3분의 1 이하로 굉장히 적어질 잠재성이 많 پای.JSONExceptionening 하나vp حد مني How Might.gravityزا Mittwoch년 وويلgruppenneجانorderby menos.ordern حي Kavavaat.havedc ежедневно tackles André المrånrası sus 亚美 lilla ↙ تؤجراء Tips ממש ينك impactgenden evenals 상세 contenu immigrants средств versatileἠϊόν rådJáΞこSSZERabol feltbi dezeیسک مماivement.tasks썰contracts momИ ALTERW 잃acjekat_prop_crfib хитឆ្នាំ.root Mindest	Arrays eineçadasב Llib prox moderسىutsuivid vedere prihod найти כולל niesтат UDPenseibase Kind تتمATICuslararası Illustr theme서비스Ẁers Variation يرى down ranged 投稿日 Hans mesmorunnerחותientation.progress'use presentationsRAровать/Auth pretende嫉interface তәм устойчив lineup რატომ filhos GleichForecast учитыGold excited көй	globalVisa čeобраз Фин:* Oxford bers вона देते birleş удостоверفية esp Wolfs Scandin


In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from dotenv import load_dotenv

load_dotenv()

# 웹 페이지 로드
loader = WebBaseLoader(
    web_paths=("https://n.news.naver.com/article/437/0000416134",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            "div",
            attrs={"class": ["newsct_article _article_body", "media_end_head go_trans" , "mdedia_end_summary"]}
        )
    )
)

# 문서 로드
docs = loader.load()

# 로드된 문서 수 확인
print(f"로드된 문서 수: {len(docs)}")

# 문서 분할
text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=20)
splits = text_splitter.split_documents(docs)

# 분할된 문서 수 확인
print(f"분할된 문서 수: {len(splits)}")

# 벡터 저장소 생성
if splits:
    vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
else:
    print("분할된 문서가 없습니다. FAISS 벡터 저장소를 생성할 수 없습니다.")

# 검색기 생성
retriever = vectorstore.as_retriever() if 'vectorstore' in locals() else None

# 질문-답변을 위한 프롬프트 템플릿 설정
prompt = PromptTemplate.from_template(
"""
당신은 질문-답변을 수행하는 AI 어시스턴트입니다.
주어진 문맥에 검색된 다음 문맥(context)을 사용해 질문에 답하세요.
만약 주어진 문맥에서 답을 찾을 수 없을 경우, 모른다고 이야기하세요.
한글로 답변해주세요.

#Question
{question}

#Context:
{context}

#Answer:"""
)

# OpenAI 모델 설정
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 질문-답변 체인 구성
if retriever:
    news_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt
        | llm
        | StrOutputParser()
    )

    # 질문 예시
    answer = news_chain.invoke("로제의 소감은 뭐래?")
    print(answer)
else:
    print("검색기를 생성할 수 없습니다.")


로드된 문서 수: 1
분할된 문서 수: 10
"세상에 무슨 일이 일어나고 있나. 이건 미쳤다"며 "넘버원(로제 팬덤), 블링크(블랙핑크 팬덤), 모든 이들에게 감사하다. 내 꿈이 이뤄졌다"라고 소감을 전했습니다.
